In [7]:
# jupyter notebook --NotebookApp.kernel_spec_manager_class='environment_kernels.EnvironmentKernelSpecManager'
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

import math
import sys, traceback
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

In [8]:
# data generating script
def place_cube(matrix, origin, r):
	#dim = matrix.shape()
	matrix[
	origin[0]-r:origin[0]+r,
	origin[1]-r:origin[1]+r,
	origin[2]-r:origin[2]+r,
	] = 1
	return matrix

def dist(x, y):
	return math.sqrt(
		(x[0]-y[0])**2+
		(x[1]-y[1])**2+
		(x[2]-y[2])**2
		)

def place_sphere(matrix, origin, r):
	for index, x in np.ndenumerate(matrix):
	    if dist(index, origin) <= r:
	    	matrix[index] = 1
	return matrix

def random_shapes(m_size,num):
    shapes = np.zeros((m_size,m_size,m_size,num))
    shape_types = np.random.randint(low = 0,high = 2,size= num)
    
    for i in range(num):
        r = np.random.randint(low = 3,high = np.floor(m_size/2-1))
        org = np.random.randint(low = r + 1,high = m_size - r - 1, size = 3)
        if shape_types[i] == 0:
            shapes[:,:,:,i] = place_cube(shapes[:,:,:,i],org,r)
        elif shape_types[i] == 1:
            shapes[:,:,:,i] = place_sphere(shapes[:,:,:,i],org,r)
            
    return shapes, shape_types
        

def matching_shapes(m_size, num):
	cubes = np.zeros((m_size,m_size,m_size,num))
	spheres = np.zeros((m_size,m_size,m_size,num))
	for i in range(num):
		r = np.random.randint(low = 3,high = np.floor(m_size/2-1))
		org = np.random.randint(low = r + 1,high = m_size - r - 1, size = 3)
		cubes[:,:,:,i] = place_cube(cubes[:,:,:,i],org,r)
		spheres[:,:,:,i] = place_sphere(spheres[:,:,:,i],org,r)
	return cubes, spheres

In [19]:
def cnn_model_fn(features, labels, mode):
  #"""Model function for CNN."""
  # Input Layer
    input_layer = tf.reshape(features, [-1, 28, 28, 1])

  # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
        

  # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

  # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    loss = None
    train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=0.001,
            optimizer="SGD")

  # Generate Predictions
    predictions = {
        "classes": tf.argmax(
          input=logits, axis=1),
        "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
    }

  # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(
        mode=mode, predictions=predictions, loss=loss, train_op=train_op)


In [22]:
def main(unused_argv):
    # Load training and eval data
    mnist = learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    mnist_classifier = learn.SKCompat(learn.Estimator(
      model_fn=cnn_model_fn, model_dir="mnist_convnet_model"))

    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

        # Train the model
    mnist_classifier.fit(
        x=train_data,
        y=train_labels,
        batch_size=100,
        steps=10,
        monitors=[logging_hook])
    
        # Configure the accuracy metric for evaluation
    metrics = {
        "accuracy":
            learn.MetricSpec(
                metric_fn=tf.metrics.accuracy, prediction_key="classes"),
    }
    
    # Evaluate the model and print results
    eval_results = mnist_classifier.score(
        x=eval_data, y=eval_labels, metrics=metrics)
    print(eval_results)



    
    
    


In [23]:
try:
    tf.app.run()
except:
    print("Exception traceback:")
    print('-'*60)
    traceback.print_exc(file=sys.stdout)
    print('-'*60)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10f591e90>, '_model_dir': 'mnist_convnet_model', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from mnist_convnet_model/model.ckpt-4300
INFO:tensorflow:Saving checkpoints fo

INFO:tensorflow:Saving checkpoints for 4310 into mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.479872.
INFO:tensorflow:Starting evaluation at 2017-06-29-12:21:14
INFO:tensorflow:Restoring parameters from mnist_convnet_model/model.ckpt-4310
INFO:tensorflow:Finished evaluation at 2017-06-29-12:21:25
INFO:tensorflow:Saving dict for global step 4310: accuracy = 0.9155, global_step = 4310, loss = 0.300374
{'loss': 0.30037364, 'global_step': 4310, 'accuracy': 0.91549999}
Exception traceback:
------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-23-a66c4a8b827d>", line 2, in <module>
    tf.app.run()
  File "/Users/rodrigo/miniconda3/envs/condaEnvPy2TF1.2/lib/python2.7/site-packages/tensorflow/python/platform/app.py", line 48, in run
    _sys.exit(main(_sys.argv[:1] + flags_passthrough))
SystemExit
------------------------------------------------------------
